In [1]:
%pylab notebook

#Pandas, numpy & Dates
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from numpy import arange

#Working directory tools
import os
from os import listdir
cwd = os.path.abspath('../')


#cwd = os.path.abspath('../..')

#Max display (monitor)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #70%

#Load custom functions
exec(open(cwd + "\\2018\\Functions.py").read())

#Stats (skewness and kurtosis)
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

from scipy import stats
from scipy import integrate

#Widgets & plots (as widgets)
import ipywidgets as widgets
from ipywidgets import ToggleButtons, VBox, HBox, HTML

from bqplot.interacts import (FastIntervalSelector, 
                              IndexSelector, BrushIntervalSelector,
                              BrushSelector, MultiSelector, 
                              LassoSelector, PanZoom, HandDraw)

#interaction
from traitlets import link
from IPython.display import HTML, display, clear_output

#File exploring window
from tkinter import Tk, filedialog 
from tkinter import *

from bqplot import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
HTML(
'''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [3]:
#Get all csv files to load the data
CsvFilesList = os.listdir(cwd + "\\2018\\Csv Files")

#Set up csv files path list
CsvFilesPath = [cwd + '\\2018\\Csv Files\\' + 
CsvFilesList[i] for i in range(0, len(CsvFilesList))]

#Dictionary key labels
DataFrameKeys = [string.replace('.csv', '') for string in CsvFilesList]

#Read data into the workspace
CLXDataList = {DataFrameKeys[i]: pd.read_csv(CsvFilesPath[i], sep = ';', 
               decimal  = ',', index_col = 0) for i in range(0, len(CsvFilesList))} 

#Date conversion & dataframe indexing by date
CLXDataList['Trades'].index = x2PyDate(CLXDataList['Trades'].index)
CLXDataList['PnL'].index = x2PyDate(CLXDataList['PnL'].index)

CLXDataList['Trades_2'].index = x2PyDate(CLXDataList['Trades_2'].index)

#FedFunds Trades
FFsTrades = CLXDataList['Trades'][CLXDataList['Trades']['Class'] == 'FF']
FFsTrades_2 = CLXDataList['Trades_2'][CLXDataList['Trades_2']['Class'] == 'FF']

#Subclasses
TradedClasses = unique(FFsTrades['SubClass']).tolist()
TradedClasses.remove('OUT')

In [4]:
#FFsTrades['2018/1'].head(5)

In [5]:
#FFsTrades_2['2019/10'].head(5)

## Meeting Exposure

In [6]:
### Data
#Exposure DF
Meeting_ExposureDf = CLXDataList['Exposure']
FactorExposureDf = CLXDataList['FactExposure'].T

#Split exposure data frames (PC and no PC)
PC_ExposureDf = CLXDataList['Exposure'].iloc[np.arange(1, 22, 2)]
NoPC_ExposureDf = CLXDataList['Exposure'].iloc[np.arange(0, 23, 2)]

In [7]:
#Exposure DF
Meeting_ExposureDf_2 = CLXDataList['Exposure_2']
FactorExposureDf_2 = CLXDataList['FactExposure_2'].T

#Split exposure data frames (PC and no PC)
PC_ExposureDf_2 = CLXDataList['Exposure_2'].iloc[np.arange(1, 22, 2)]
NoPC_ExposureDf_2 = CLXDataList['Exposure_2'].iloc[np.arange(0, 23, 2)]

FullFactorExposureDf = pd.concat([FactorExposureDf, FactorExposureDf_2])

In [8]:
#Plot data
############################################################
x_data = PC_ExposureDf.columns.tolist()
y_data = PC_ExposureDf.index.tolist()
MapData = np.array(PC_ExposureDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
x_sc = OrdinalScale()
y_sc = OrdinalScale(reverse = True)
col_sc = ColorScale( colors = ['#DE2369', 
                              '#EDEDED',
                              '#01B0F0'])

ax_x = Axis(scale = x_sc, 
            label = 'Meeting',
            label_location = 'end',
            label_offset = '50px') 

ax_y = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label = 'Month',
            label_location = 'end',
            label_offset = '50px')

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Exposure (USD)', 
                   tick_format = '$0,.0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = x_data, 
                       column = y_data, 
                       color = MapData, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['row', 
                           'column', 'color'], 
                 formats = ['', '', '$0,.0f'], 
                 labels = ['Hike', 'Month', 
                           'Exposure'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure20 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y, col_ax])

#Figure Title
figure20.title = '2018 - Exposure HeatMap, PC Meetings (Read from top row to bottom)'

#Figure layout (except size attributes, defined on section 4)
figure20.title_style = {'font-size': '13px'}
figure20.background_style = {'fill': 'white'}
figure20.animation_duration = 500 

figure20.fig_margin = {'top':40, 'bottom':100, 'left':60, 'right':0}
figure20.layout.width = '35%'
figure20.layout.left = '200px'
figure20.layout.height = '700px'

In [9]:
#Plot data
############################################################
x_data = NoPC_ExposureDf.columns.tolist()
y_data = NoPC_ExposureDf.index.tolist()
MapData = np.array(NoPC_ExposureDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
x_sc = OrdinalScale()
y_sc = OrdinalScale(reverse = True)
col_sc = ColorScale(min = float(np.min(np.array(PC_ExposureDf.T))), 
                    max = float(np.max(np.array(PC_ExposureDf.T))), 
    colors = ['#DE2369', '#EDEDED', '#01B0F0'])

ax_x = Axis(scale = x_sc, 
            label = 'Meeting',
            label_location = 'end',
            label_offset = '50px') 

ax_y = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label = 'Month',
            label_location = 'end',
            label_offset = '50px')

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Exposure (USD)', 
                   tick_format = '$0,.0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = x_data, 
                       column = y_data, 
                       color = MapData, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['row', 
                           'column', 'color'], 
                 formats = ['', '', '$0,.0f'], 
                 labels = ['Hike', 'Month', 
                           'Exposure'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure21 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y, col_ax])

#Figure Title
figure21.title = '2018 - Exposure HeatMap, NoPC Meetings (Read from top row to bottom)'

#Figure layout (except size attributes, defined on section 4)
figure21.title_style = {'font-size': '13px'}
figure21.background_style = {'fill': 'white'}
figure21.animation_duration = 500 

figure21.fig_margin = {'top':40, 'bottom':100, 'left':60, 'right':0}
figure21.layout.width = '35%'
figure21.layout.left = '200px'
figure21.layout.height = '700px'

In [10]:
#Plot data
############################################################
x_data = PC_ExposureDf_2.columns.tolist()
y_data = PC_ExposureDf_2.index.tolist()
MapData = np.array(PC_ExposureDf_2.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
x_sc = OrdinalScale()
y_sc = OrdinalScale(reverse = True)
col_sc =ColorScale(min = float(np.min(np.array(PC_ExposureDf.T))), 
                   max = float(np.max(np.array(PC_ExposureDf.T))), 
    colors = ['#DE2369', '#EDEDED', '#01B0F0'])

ax_x = Axis(scale = x_sc, 
            label = 'Meeting',
            label_location = 'end',
            label_offset = '50px') 

ax_y = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label = 'Month',
            label_location = 'end',
            label_offset = '50px')

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Exposure (USD)', 
                   tick_format = '$0,.0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = x_data, 
                       column = y_data, 
                       color = MapData, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['row', 
                           'column', 'color'], 
                 formats = ['', '', '$0,.0f'], 
                 labels = ['Hike', 'Month', 
                           'Exposure'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure20_2 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y, col_ax])

#Figure Title
figure20_2.title = '2019 - Exposure HeatMap, PC Meetings (Read from top row to bottom)'

#Figure layout (except size attributes, defined on section 4)
figure20_2.title_style = {'font-size': '13px'}
figure20_2.background_style = {'fill': 'white'}
figure20_2.animation_duration = 500 

figure20_2.fig_margin = {'top':40, 'bottom':100, 'left':60, 'right':0}
figure20_2.layout.width = '35%'
figure20_2.layout.left = '250px'
figure20_2.layout.height = '700px'

In [11]:
#Plot data
############################################################
x_data = NoPC_ExposureDf_2.columns.tolist()
y_data = NoPC_ExposureDf_2.index.tolist()
MapData = np.array(NoPC_ExposureDf_2.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
x_sc = OrdinalScale()
y_sc = OrdinalScale(reverse = True)
col_sc = ColorScale(min = float(np.min(np.array(PC_ExposureDf.T))), 
                    max = float(np.max(np.array(PC_ExposureDf.T))), 
    colors = ['#DE2369', '#EDEDED', '#01B0F0'])

ax_x = Axis(scale = x_sc, 
            label = 'Meeting',
            label_location = 'end',
            label_offset = '50px') 

ax_y = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label = 'Month',
            label_location = 'end',
            label_offset = '50px')

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Exposure (USD)', 
                   tick_format = '$0,.0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = x_data, 
                       column = y_data, 
                       color = MapData, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['row', 
                           'column', 'color'], 
                 formats = ['', '', '$0,.0f'], 
                 labels = ['Hike', 'Month', 
                           'Exposure'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure21_2 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y, col_ax])

#Figure Title
figure21_2.title = '2019 - Exposure HeatMap, NoPC Meetings (Read from top row to bottom)'

#Figure layout (except size attributes, defined on section 4)
figure21_2.title_style = {'font-size': '13px'}
figure21_2.background_style = {'fill': 'white'}
figure21_2.animation_duration = 500 

figure21_2.fig_margin = {'top':40, 'bottom':100, 'left':60, 'right':0}
figure21_2.layout.width = '35%'
figure21_2.layout.left = '250px'
figure21_2.layout.height = '700px'

In [12]:
HBox([figure20, figure20_2])

In [13]:
HBox([figure21, figure21_2])

In [14]:
#Plot data
############################################################
x_data = FullFactorExposureDf.index.tolist()
y_data = np.array(FullFactorExposureDf.iloc[:, 0]).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale()  

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '13px'}, 
                   num_ticks = 10)

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '$0,.0f', 
                   tick_style = {'font-size': '13px'},
                   grid_color = '#EDEDED')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['Level'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#DE2369'], 
               stroke_width = 3, 
               visible = True, 
               display_legend = True, 
               )

PointsScatt = Scatter(x = np.array(x_data), 
                      y = y_data,  
                      scales = {'x': MainTs_x_dates, 
                                'y' : MainTs_y_sc},                      
                     padding = 0.25, colors = ['#01B0F0'], 
                     default_size = 100, 
                     default_opacities = [0.6], 
                     unhovered_style = {'opacity': 0.2}, 
                     hovered_style = {'opacity': 0.8})
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['', '$0,.0f'], 
         labels = ['Month', 'Exposure'])

#Set up the tooltip interaction 
PointsScatt.tooltip = def_tt
PointsScatt.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure22 = Figure(marks = [TsPlot, PointsScatt], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure22.title = 'Press Conference Factor Exposure Evolution - LEVEL'

#Figure layout (except size attributes, defined on section 4)
figure22.title_style = {'font-size': '13px'}
figure22.background_style = {'fill': 'white'}
figure22.animation_duration = 800 
    
figure22.fig_margin = {'top':40, 'bottom':60, 'left':100, 'right':10}
figure22.layout.width = '45%'
figure22.layout.height = '500px'
figure22.legend_location = 'top-left'

In [15]:
#Plot data
############################################################
x_data = FullFactorExposureDf.index.tolist()
y_data = np.array(FullFactorExposureDf.iloc[:, 1]).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale()  

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '13px'}, 
                   num_ticks = 10)

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '$0,.0f', 
                   tick_style = {'font-size': '13px'},
                   grid_color = '#EDEDED')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['Slope'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0'], 
               stroke_width = 3, 
               visible = True, 
               display_legend = True, 
               )

PointsScatt = Scatter(x = np.array(x_data), 
                      y = y_data,  
                      scales = {'x': MainTs_x_dates, 
                                'y' : MainTs_y_sc},                      
                     padding = 0.25, colors = ['#DE2369'], 
                     default_size = 100, 
                     default_opacities = [0.6], 
                     unhovered_style = {'opacity': 0.2}, 
                     hovered_style = {'opacity': 0.8})
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['', '$0,.0f'], 
         labels = ['Month', 'Exposure'])

#Set up the tooltip interaction 
PointsScatt.tooltip = def_tt
PointsScatt.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure22_2 = Figure(marks = [TsPlot, PointsScatt], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure22_2.title = 'Press Conference Factor Exposure Evolution - SLOPE'

#Figure layout (except size attributes, defined on section 4)
figure22_2.title_style = {'font-size': '13px'}
figure22_2.background_style = {'fill': 'white'}
figure22_2.animation_duration = 800 
    
figure22_2.fig_margin = {'top':40, 'bottom':60, 'left':100, 'right':10}
figure22_2.layout.width = '45%'
figure22_2.layout.height = '500px'
figure22_2.legend_location = 'top-left'

In [16]:
#Plot data
############################################################
x_data = FullFactorExposureDf.index.tolist()
y_data = np.array(FullFactorExposureDf.iloc[:, 2]).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale()  

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '13px'}, 
                   num_ticks = 10)

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '$0,.0f', 
                   tick_style = {'font-size': '13px'},
                   grid_color = '#EDEDED')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['Level'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#DE2369', '#01B0F0'], 
               stroke_width = 3, 
               visible = True, 
               display_legend = True, 
               )

PointsScatt = Scatter(x = np.array(x_data), 
                      y = y_data,  
                      scales = {'x': MainTs_x_dates, 
                                'y' : MainTs_y_sc},                      
                     padding = 0.25, colors = ['#01B0F0'], 
                     default_size = 100, 
                     default_opacities = [0.6], 
                     unhovered_style = {'opacity': 0.2}, 
                     hovered_style = {'opacity': 0.8})
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['', '$0,.0f'], 
         labels = ['Month', 'Exposure'])

#Set up the tooltip interaction 
PointsScatt.tooltip = def_tt
PointsScatt.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure23 = Figure(marks = [TsPlot, PointsScatt], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure23.title = 'No Press Conference Factor Exposure Evolution - LEVEL'

#Figure layout (except size attributes, defined on section 4)
figure23.title_style = {'font-size': '13px'}
figure23.background_style = {'fill': 'white'}
figure23.animation_duration = 800 
    
figure23.fig_margin = {'top':40, 'bottom':60, 'left':100, 'right':10}
figure23.layout.width = '45%'
figure23.layout.left = '0px'
figure23.layout.height = '500px'
figure23.legend_location = 'top-left'

In [17]:
#Plot data
############################################################
x_data = FullFactorExposureDf.index.tolist()
y_data = np.array(FullFactorExposureDf.iloc[:, 3]).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale()  

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '13px'}, 
                   num_ticks = 10)

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '$0,.0f', 
                   tick_style = {'font-size': '13px'},
                   grid_color = '#EDEDED')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['Slope'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0'], 
               stroke_width = 3, 
               visible = True, 
               display_legend = True, 
               )

PointsScatt = Scatter(x = np.array(x_data), 
                      y = y_data,  
                      scales = {'x': MainTs_x_dates, 
                                'y' : MainTs_y_sc},                      
                     padding = 0.25, colors = ['#DE2369'], 
                     default_size = 100, 
                     default_opacities = [0.6], 
                     unhovered_style = {'opacity': 0.2}, 
                     hovered_style = {'opacity': 0.8})
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['', '$0,.0f'], 
         labels = ['Month', 'Exposure'])

#Set up the tooltip interaction 
PointsScatt.tooltip = def_tt
PointsScatt.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure23_2 = Figure(marks = [TsPlot, PointsScatt], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure23_2.title = 'No Press Conference Factor Exposure Evolution - SLOPE'

#Figure layout (except size attributes, defined on section 4)
figure23_2.title_style = {'font-size': '13px'}
figure23_2.background_style = {'fill': 'white'}
figure23_2.animation_duration = 800 
    
figure23_2.fig_margin = {'top':40, 'bottom':60, 'left':100, 'right':10}
figure23_2.layout.width = '45%'
figure23_2.layout.left = '0px'
figure23_2.layout.height = '500px'
figure23_2.legend_location = 'top-left'

In [18]:
VBox([HBox([figure22, figure23]), HBox([figure22_2, figure23_2])])

## PnL Summary: PnL per Type of Strategy (Cal1M, Cal2M, Fly1M, Cdr1M, Cdr2M)

In [19]:
### Data

#Split data frames by class
Dfs_byClass = {key: SplitDf for key, SplitDf in FFsTrades.groupby('SubClass')}

#Total pnl per strategy df
PnL_byClass = (pd.DataFrame(data = np.array([sum(Dfs_byClass[Tclass]['NetPnL']) for 
                                                 Tclass in TradedClasses]), 
                            index = TradedClasses, columns = ['PnL']))

#Total pnl split by PC/noPC 
PnL_byHikeClass = pd.DataFrame(data = np.array([PnL_byClass.iloc[1] + PnL_byClass.iloc[3], 
                                               PnL_byClass.iloc[0] + PnL_byClass.iloc[2] + PnL_byClass.iloc[4]]),
                              index = ['Pc', 'NoPc'], columns = ['PnL'])

In [20]:
#2019

#Split data frames by class
Dfs_byClass_2 = {key: SplitDf for key, SplitDf in FFsTrades_2.groupby('SubClass')}

#Total pnl per strategy df
PnL_byClass_2 = (pd.DataFrame(data = np.array([sum(Dfs_byClass_2[Tclass]['NetPnL']) for 
                                                 Tclass in TradedClasses]), 
                            index = TradedClasses, columns = ['PnL']))

#Total pnl split by PC/noPC 
PnL_byHikeClass_2 = pd.DataFrame(data = np.array([PnL_byClass_2.iloc[1] + PnL_byClass_2.iloc[3], 
                                               PnL_byClass_2.iloc[0] + PnL_byClass_2.iloc[2] + PnL_byClass_2.iloc[4]]),
                              index = ['Pc', 'NoPc'], columns = ['PnL'])

In [21]:
#Plot data
############################################################
ClassPnL = np.array([PnL_byClass['PnL'], PnL_byClass_2['PnL']])
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '14px'})

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            orientation = 'vertical',
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '12px'},
            grid_color = '#EDEDED')
############################################################  

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = TradedClasses, 
           y = ClassPnL,
           labels = ['2018', '2019'],
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#3CDBCC', '#DE2369'],
           padding = 0.25,
           type = 'grouped',
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True, display_legend = True)  
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Ticker', 
                           'PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure3 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure3.title = 'Closed PnL by Type of Strategy (USD)'

#Figure layout (except size attributes, defined on section 4)
figure3.title_style = {'font-size': '13px'}
figure3.background_style = {'fill': 'white'}
figure3.animation_duration = 800  

figure3.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure3.layout.height = '450px'
figure3.legend_location = 'top-left'
figure3.layout.left = '140px'
figure3.layout.width = '50%'

In [22]:
### Data

#Traded Strategies
TradedStrategies = unique(FFsTrades['Strategy']).tolist()

#Split data frames by strategy 
Dfs_byTicker = {key: SplitDf for key, SplitDf in FFsTrades.groupby('Strategy')}

#Total pnl per strategy df
PnL_byTicker = (pd.DataFrame(data = np.array([sum(Dfs_byTicker[Ticker]['NetPnL']) for 
                                              Ticker in TradedStrategies]), 
                             index = TradedStrategies, columns = ['PnL']))

#Top 10 strategies by pnl
TopStrats = PnL_byTicker.nlargest(15, columns = 'PnL')
BotStrats = PnL_byTicker.nsmallest(15, columns = 'PnL')

In [23]:
#2019
#Traded Strategies
TradedStrategies_2 = unique(FFsTrades_2['Strategy']).tolist()

#Split data frames by strategy 
Dfs_byTicker_2 = {key: SplitDf for key, SplitDf in FFsTrades_2.groupby('Strategy')}

#Total pnl per strategy df
PnL_byTicker_2 = (pd.DataFrame(data = np.array([sum(Dfs_byTicker_2[Ticker]['NetPnL']) for 
                                              Ticker in TradedStrategies_2]), 
                             index = TradedStrategies_2, columns = ['PnL']))

#Top 10 strategies by pnl
TopStrats_2 = PnL_byTicker_2.nlargest(15, columns = 'PnL')
BotStrats_2 = PnL_byTicker_2.nsmallest(15, columns = 'PnL')

In [24]:
#Plot data
############################################################
PnLStrats = TopStrats.sort_values(by = 'PnL', ascending = True)

x_data = PnLStrats.index.tolist()
y_data = np.array(PnLStrats)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#01B0F0'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Ticker', 
                           'PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure4 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure4.title = '2018 - Top Most Profitable Strategies'

#Figure layout (except size attributes, defined on section 4)
figure4.title_style = {'font-size': '13px'}
figure4.background_style = {'fill': 'white'}
figure4.animation_duration = 800  

figure4.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure4.layout.width = '30%'
figure4.layout.height = '450px'
figure4.layout.left = '20px'
figure4.legend_location = 'top-left'

In [25]:
#Plot data
############################################################
PnLStrats_2 = TopStrats_2.sort_values(by = 'PnL', ascending = True)

x_data = PnLStrats_2.index.tolist()
y_data = np.array(PnLStrats_2)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#DE2369'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Ticker', 
                           'PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure4_2 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure4_2.title = '2019 - Top Most Profitable Strategies' 

#Figure layout (except size attributes, defined on section 4)
figure4_2.title_style = {'font-size': '13px'}
figure4_2.background_style = {'fill': 'white'}
figure4_2.animation_duration = 800  

figure4_2.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure4_2.layout.width = '30%'
figure4_2.layout.height = '450px'
figure4_2.layout.left = '20px'
figure4_2.legend_location = 'top-left'

In [26]:
#Plot data
############################################################
PnLStrats_3 = BotStrats.sort_values(by = 'PnL', ascending = True)

x_data = PnLStrats_3.index.tolist()
y_data = np.array(PnLStrats_3)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#01B0F0'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Ticker', 
                           'PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure4_2_1 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure4_2_1.title = '2018 - Top Least Profitable Strategies' 

#Figure layout (except size attributes, defined on section 4)
figure4_2_1.title_style = {'font-size': '13px'}
figure4_2_1.background_style = {'fill': 'white'}
figure4_2_1.animation_duration = 800  

figure4_2_1.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure4_2_1.layout.width = '30%'
figure4_2_1.layout.height = '450px'
figure4_2_1.legend_location = 'top-left'

In [27]:
#Plot data
############################################################
PnLStrats_4 = BotStrats_2.sort_values(by = 'PnL', ascending = True)

x_data = PnLStrats_4.index.tolist()
y_data = np.array(PnLStrats_4)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#DE2369'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Ticker', 
                           'PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure4_2_2 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure4_2_2.title = '2019 - Top Least Profitable Strategies'

#Figure layout (except size attributes, defined on section 4)
figure4_2_2.title_style = {'font-size': '13px'}
figure4_2_2.background_style = {'fill': 'white'}
figure4_2_2.animation_duration = 800  

figure4_2_2.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure4_2_2.layout.width = '30%'
figure4_2_2.layout.height = '450px'
figure4_2_2.legend_location = 'top-left'

In [28]:
figure3

Figure(animation_duration=800, axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '14px'}), Axis(grid_c…

## PnL Summary: PnL per Specific Strategy

In [29]:
HBox([figure4, figure4_2])

In [30]:
HBox([figure4_2_1, figure4_2_2])

## Closed Trades Summary: Number of Closed Trades per Class (PC, noPC) and Subclass (Cal1M, Cal2M, Fly1M, Cdr1M, Cdr2M)

In [31]:
### Data

#Number of trades per strategy df
Ntrades_byClass = (pd.DataFrame(data = np.array([len(Dfs_byClass[Tclass]) for 
                                                Tclass in TradedClasses]), 
                                index = TradedClasses, columns = ['NTrades']))

#Number of trades split by PC/noPC 
Ntrades_byHikeClass = pd.DataFrame(data = np.array([Ntrades_byClass.iloc[1] + Ntrades_byClass.iloc[3], 
                                               Ntrades_byClass.iloc[0] + Ntrades_byClass.iloc[2] + Ntrades_byClass.iloc[4]]),
                              index = ['Pc', 'NoPc'], columns = ['NTrades'])

In [32]:
#Number of trades per strategy df
Ntrades_byClass_2 = (pd.DataFrame(data = np.array([len(Dfs_byClass_2[Tclass]) for 
                                                Tclass in TradedClasses]), 
                                index = TradedClasses, columns = ['NTrades']))

#Number of trades split by PC/noPC 
Ntrades_byHikeClass_2 = pd.DataFrame(data = np.array([Ntrades_byClass_2.iloc[1] + Ntrades_byClass_2.iloc[3], 
                                               Ntrades_byClass_2.iloc[0] + Ntrades_byClass_2.iloc[2] + Ntrades_byClass_2.iloc[4]]),
                              index = ['Pc', 'NoPc'], columns = ['NTrades'])

In [33]:
#Plot data
############################################################
HikeClassNTrades = np.array(Ntrades_byHikeClass['NTrades'])

HikeClassNTLabels = ([str(k) + ': ' +  
                     str(float(round(Ntrades_byHikeClass.loc[k] /   
                     sum(Ntrades_byHikeClass)  * 100, 0))) +   
                     '%' for k in ['Pc', 'NoPc']])

HikeClassNTrades_2 = np.array(Ntrades_byHikeClass_2['NTrades'])

HikeClassNTLabels_2 = ([str(k) + ': ' +  
                     str(float(round(Ntrades_byHikeClass_2.loc[k] /   
                     sum(Ntrades_byHikeClass_2)  * 100, 0))) +   
                     '%' for k in ['Pc', 'NoPc']])

############################################################
  
#Step 2: Marks (plots)
############################################################
pie = Pie(sizes = HikeClassNTrades, 
           inner_radius = 100, 
           labels = HikeClassNTLabels, 
           colors = ['#01B0F0', '#DE2369'],           
           stroke = 'white', sort = True, 
           font_size = '12px') 

pie2 = Pie(sizes = HikeClassNTrades_2, 
           inner_radius = 100, 
           labels = HikeClassNTLabels_2, 
           colors = ['#01B0F0', '#DE2369'],           
           stroke = 'white', sort = True, 
           font_size = '12px') 
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['label', 'size'], 
                 formats = ['', '0f'], 
                 labels = ['label', 'Qx Closed Trades'])

pie.tooltip = def_tt
pie.interactions = {'hover': 'tooltip'}
pie2.tooltip = def_tt
pie2.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure6 = Figure(marks = [pie])

#Figure Title
figure6.title = ('2018 Number of Closed Trades by Type of Hike, out of ' + 
                 str(int(sum(Ntrades_byHikeClass))) + ' Trades')

#Figure layout (except size attributes, defined on section 4)
figure6.title_style = {'font-size': '13px'}
figure6.background_style = {'fill': 'white'}
figure6.animation_duration = 800  

figure6.fig_margin = {'top':0, 'bottom':0, 'left':0, 'right':0}
figure6.layout.width = '23%'
figure6.layout.height = '450px'

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure6_2 = Figure(marks = [pie2])

#Figure Title
figure6_2.title = ('2019 Number of Closed Trades by Type of Hike, out of ' + 
                 str(int(sum(Ntrades_byHikeClass_2))) + ' Trades')

#Figure layout (except size attributes, defined on section 4)
figure6_2.title_style = {'font-size': '13px'}
figure6_2.background_style = {'fill': 'white'}
figure6_2.animation_duration = 800  

figure6_2.fig_margin = {'top':0, 'bottom':0, 'left':0, 'right':0}
figure6_2.layout.width = '23%'
figure6_2.layout.height = '450px'

In [34]:
#Plot data
############################################################
ClassNTrades = np.array([Ntrades_byClass['NTrades'], Ntrades_byClass_2['NTrades']])
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '14px'})

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            orientation = 'vertical',
            tick_format = '0f', 
            tick_style = {'font-size': '12px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = TradedClasses, 
           y = ClassNTrades, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           labels = ['2018', '2019'],
           type = 'grouped',
           colors = ['#3CDBCC', '#DE2369'],
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True, display_legend = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '0f'], 
                 labels = ['Ticker', 
                           'Number of Trades'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure7.title = 'Number of Trades by Type of Strategy'

#Figure layout (except size attributes, defined on section 4)
figure7.title_style = {'font-size': '13px'}
figure7.background_style = {'fill': 'white'}
figure7.animation_duration = 800  

figure7.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure7.layout.width = '43%'
figure7.layout.height = '450px'
figure7.legend_location = 'top-left'

In [35]:
HBox([figure6, figure6_2, figure7])

## Closed Trades Summary: Number of Closed Trades per Specific Strategy

In [36]:
### Data
#Total pnl per strategy df
NTrades_byTicker = (pd.DataFrame(data = np.array([len(Dfs_byTicker[Ticker]) for 
                                                 Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['NTrades']))

#Top 10 strategies by Qx
TopNTStrats = NTrades_byTicker.nlargest(15, columns = 'NTrades')

In [37]:
#Total pnl per strategy df
NTrades_byTicker_2 = (pd.DataFrame(data = np.array([len(Dfs_byTicker_2[Ticker]) for 
                                                 Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['NTrades']))

#Top 10 strategies by Qx
TopNTStrats_2 = NTrades_byTicker_2.nlargest(15, columns = 'NTrades')

In [38]:
#Plot data
############################################################
NTradesStrats = TopNTStrats.sort_values(by = 'NTrades', 
                                        ascending = True)

x_data = NTradesStrats.index.tolist()
y_data = np.array(NTradesStrats)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#01B0F0'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.0f'], 
                 labels = ['Ticker', 
                           'Number of Trades'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure8 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure8.title = ('2018 - Top Most Traded Strategies' + 
                 ' (' + str(int(round(float(sum(y_data) / 
                 sum(np.array(NTrades_byTicker['NTrades']))) * 100, 0))) + 
                 '% of Total Trades' + ')')

#Figure layout (except size attributes, defined on section 4)
figure8.title_style = {'font-size': '13px'}
figure8.background_style = {'fill': 'white'}
figure8.animation_duration = 800  

figure8.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure8.layout.width = '30%'
figure8.layout.height = '450px'
figure8.legend_location = 'top-left'

In [39]:
#Plot data
############################################################
NTradesStrats_2 = TopNTStrats_2.sort_values(by = 'NTrades', 
                                        ascending = True)

x_data = NTradesStrats_2.index.tolist()
y_data = np.array(NTradesStrats_2)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '12px'},
            orientation = 'vertical')

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '0f', 
            tick_style = {'font-size': '11px'},
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#DE2369'], #'#F05471'],
           orientation = 'horizontal',
           padding = 0.25,  
           stroke = '#D8D8D8', 
           selected_style = {'opacity': 0.4},
           visible = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.0f'], 
                 labels = ['Ticker', 
                           'Number of Trades'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure8_2 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure8_2.title = ('2019 - Top Most Traded Strategies' + 
                 ' (' + str(int(round(float(sum(y_data) / 
                 sum(np.array(NTrades_byTicker_2['NTrades']))) * 100, 0))) + 
                 '% of Total Trades' + ')')

#Figure layout (except size attributes, defined on section 4)
figure8_2.title_style = {'font-size': '13px'}
figure8_2.background_style = {'fill': 'white'}
figure8_2.animation_duration = 800  

figure8_2.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure8_2.layout.width = '30%'
figure8_2.layout.height = '450px'
figure8_2.legend_location = 'top-left'

In [40]:
HBox([figure8, figure8_2])

## Trades Overview Summary:  PnL & Number of Trades per Subclass (Cal1M, Cal2M, Fly1M, Cdr1M, Cdr2M) 

In [41]:
### Data

TimespanStackedB = ['2017/12', '2018/1', '2018/2', '2018/3', 
                    '2018/4', '2018/5', '2018/6', '2018/7', 
                    '2018/8', '2018/9', '2018/10', '2018/11']

TimespanStackedB_1 = ['2017/12', '2018/3', '2018/6', '2018/9']
TimespanStackedB_2 = ['2018/2', '2018/5', '2018/8', '2018/11']

#2019
TimespanStackedB_1_2 = ['2018/12', '2019/3', '2019/6', '2019/9']
TimespanStackedB_2_2 = ['2019/2', '2019/5', '2019/8', '2019/11']

#Monthly pnl per class (Cals & flies)
Cal1M_MonthlyPnL = [sum(np.array(Dfs_byClass['Cal1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cal2M_MonthlyPnL = [sum(np.array(Dfs_byClass['Cal2M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cdr1M_MonthlyPnL = [sum(np.array(Dfs_byClass['Cdr1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cdr2M_MonthlyPnL = [sum(np.array(Dfs_byClass['Cdr2M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Fly1M_MonthlyPnL = [sum(np.array(Dfs_byClass['Fly1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]

Cal1M_MonthlyPnL_2 = [sum(np.array(Dfs_byClass_2['Cal1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cal2M_MonthlyPnL_2 = [sum(np.array(Dfs_byClass_2['Cal2M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cdr1M_MonthlyPnL_2 = [sum(np.array(Dfs_byClass_2['Cdr1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Cdr2M_MonthlyPnL_2 = [sum(np.array(Dfs_byClass_2['Cdr2M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]
Fly1M_MonthlyPnL_2 = [sum(np.array(Dfs_byClass_2['Fly1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['NetPnL'])) for 
                    i in range(0, 4)]

#Monthly pnl per class (Cals & flies) DF
MonthlyPnLDf = pd.DataFrame(data = np.array((Cal1M_MonthlyPnL + Cal1M_MonthlyPnL_2, 
                                             Cal2M_MonthlyPnL + Cal2M_MonthlyPnL_2, 
                                             Cdr1M_MonthlyPnL + Cdr1M_MonthlyPnL_2,
                                             Cdr2M_MonthlyPnL + Cdr2M_MonthlyPnL_2,
                                             Fly1M_MonthlyPnL + Fly1M_MonthlyPnL_2)).T, 
                            index = ['Q' + str(i) for i in range(1, 9)],
                            #index = PC_ExposureDf.columns.tolist(), 
                            columns = TradedClasses)

#Monthly number of trades per class (Cals & flies)
Cal1M_MonthlyQx = [sum(abs(np.array(Dfs_byClass['Cal1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cal2M_MonthlyQx = [sum(abs(np.array(Dfs_byClass['Cal2M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cdr1M_MonthlyQx = [sum(abs(np.array(Dfs_byClass['Cdr1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cdr2M_MonthlyQx = [sum(abs(np.array(Dfs_byClass['Cdr2M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Fly1M_MonthlyQx = [sum(abs(np.array(Dfs_byClass['Fly1M'][TimespanStackedB_1[i]:TimespanStackedB_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]

#2019
Cal1M_MonthlyQx_2 = [sum(abs(np.array(Dfs_byClass_2['Cal1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cal2M_MonthlyQx_2 = [sum(abs(np.array(Dfs_byClass_2['Cal2M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cdr1M_MonthlyQx_2 = [sum(abs(np.array(Dfs_byClass_2['Cdr1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Cdr2M_MonthlyQx_2 = [sum(abs(np.array(Dfs_byClass_2['Cdr2M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]
Fly1M_MonthlyQx_2 = [sum(abs(np.array(Dfs_byClass_2['Fly1M'][TimespanStackedB_1_2[i]:TimespanStackedB_2_2[i]]['TotalQty']))) for 
                    i in range(0, 4)]

#Monthly number of trades per class (Cals & flies) DF
MonthlyQxDf = pd.DataFrame(data = np.array((Cal1M_MonthlyQx + Cal1M_MonthlyQx_2, 
                                             Cal2M_MonthlyQx + Cal2M_MonthlyQx_2,                
                                             Cdr1M_MonthlyQx + Cdr1M_MonthlyQx_2,
                                             Cdr2M_MonthlyQx + Cdr2M_MonthlyQx_2,
                                             Fly1M_MonthlyQx + Fly1M_MonthlyQx_2)).T, 
                            index = ['Q' + str(i) for i in range(1, 9)], 
                            #index = PC_ExposureDf.columns.tolist(), 
                            columns = TradedClasses)

In [42]:
#Plot data
############################################################
x_data = MonthlyPnLDf.index.tolist()
y_data = np.array(MonthlyPnLDf).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '13px'})

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '13px'},
            grid_color = '#EDEDED',
            orientation = 'vertical')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           labels = TradedClasses, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#0070C0', '#01B0F0',  
                    '#3CDBCC', '#31DC8D',
                    '#DE2369', '#F53F52',
                    '#F05471'],
           type = 'stacked', 
           padding = 0.25,  
           stroke = 'black', 
           selected_style = {'opacity': 0.4},
           visible = True, 
           display_legend = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '$0,.0f'], 
                 labels = ['Month', 
                           'Closed PnL'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure24 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure24.title = '2018/2019 - Closed PnL by Type of Strategy per Quarter'

#Figure layout (except size attributes, defined on section 4)
figure24.title_style = {'font-size': '13px'}
figure24.background_style = {'fill': 'white'}
figure24.animation_duration = 800  

figure24.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure24.layout.width = '45%'
figure24.layout.height = '700px'
figure24.layout.left = '50px'
figure24.legend_location = 'top-right'

In [43]:
#Plot data
############################################################
x_data = MonthlyQxDf.index.tolist()
y_data = np.array(MonthlyQxDf).T
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Bar_x = OrdinalScale()
Bar_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Bar_x, 
            tick_style = {'font-size': '13px'})

#Y-Axis
ax_y = Axis(scale = Bar_y, 
            tick_format = '0f', 
            tick_style = {'font-size': '13px'},
            grid_color = '#EDEDED',
            orientation = 'vertical')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes cross section bar plot
bar = Bars(x = x_data, 
           y = y_data, 
           labels = TradedClasses, 
           scales={'x': Bar_x, 
                   'y':Bar_y},
           colors = ['#0070C0', '#01B0F0',  
                    '#3CDBCC', '#31DC8D',
                    '#DE2369', '#F53F52',
                    '#F05471'],
           type = 'stacked', 
           padding = 0.25,  
           stroke = 'black', 
           selected_style = {'opacity': 0.4},
           visible = True, 
           display_legend = True)        
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '0f'], 
                 labels = ['Quarter', 
                           'Number of Trades'])

#Set up the tooltip interaction 
bar.tooltip = def_tt
bar.interactions = {'hover': 'tooltip', 
                    'click': 'select'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure25 = Figure(marks = [bar], axes = [ax_x, ax_y])

#Figure Title
figure25.title = '2018/2019 - Number of Units traded by Type of Strategy per Quarter'

#Figure layout (except size attributes, defined on section 4)
figure25.title_style = {'font-size': '13px'}
figure25.background_style = {'fill': 'white'}
figure25.animation_duration = 800  

figure25.fig_margin = {'top':40, 'bottom':30, 'left':80, 'right':16}
figure25.layout.width = '45%'
figure25.layout.height = '700px'
figure25.layout.left = '100px'
figure25.legend_location = 'top-right'

In [44]:
HBox([figure24, figure25])

## PnL and Number of Trades per Strategy and Subclass (combined)

In [45]:
### Data
#Strategy subclass
SubClass_byTicker = (pd.DataFrame(data = np.array([unique(Dfs_byTicker[Ticker]['SubClass']) for 
                                                   Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['SubClass']))

#Top trades (most PnL and number of trades)
NTrades_TopPerfStr = NTrades_byTicker.loc[(
                     unique(TopStrats.index.tolist() + 
                            TopNTStrats.index.tolist()).tolist())]

PnL_TopPerfStr = PnL_byTicker.loc[(
                 unique(TopStrats.index.tolist() + 
                        TopNTStrats.index.tolist()).tolist())] 

In [46]:
#Strategy subclass
SubClass_byTicker_2 = (pd.DataFrame(data = np.array([unique(Dfs_byTicker_2[Ticker]['SubClass']) for 
                                                   Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['SubClass']))

#Top trades (most PnL and number of trades)
NTrades_TopPerfStr_2 = NTrades_byTicker_2.loc[(
                     unique(TopStrats_2.index.tolist() + 
                            TopNTStrats_2.index.tolist()).tolist())]

PnL_TopPerfStr_2 = PnL_byTicker_2.loc[(
                 unique(TopStrats_2.index.tolist() + 
                        TopNTStrats_2.index.tolist()).tolist())] 

In [47]:
#Plot data
############################################################
Scatt_x_data = np.array(NTrades_byTicker['NTrades'])
Scatt_y_data = np.array(PnL_byTicker['PnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(NTrades_TopPerfStr['NTrades'])
Label_y_data = np.array(PnL_TopPerfStr['PnL'])
Label_data = NTrades_TopPerfStr.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Trades',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'PnL (USD)', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale=Scatt_o, label='Class', 
                 side='right', orientation='vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '$0,.0f'], 
                 labels = ['Strategy', 'NTrades', 
                           'PnL'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure9 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure9.title = '2018 - Strategies by Number of Trades and Closed PnL (size by PnL)'

#Figure layout (except size attributes, defined on section 4) 
figure9.title_style = {'font-size': '13px'}
figure9.background_style = {'fill': 'white'} 
figure9.animation_duration = 100

figure9.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure9.layout.width = '48%' #98%
figure9.layout.height = '900px'
figure9.legend_location = 'top-left'

In [48]:
#Plot data
############################################################
Scatt_x_data = np.array(NTrades_byTicker_2['NTrades'])
Scatt_y_data = np.array(PnL_byTicker_2['PnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(NTrades_TopPerfStr_2['NTrades'])
Label_y_data = np.array(PnL_TopPerfStr_2['PnL'])
Label_data = NTrades_TopPerfStr_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Trades',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'PnL (USD)', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale=Scatt_o, label='Class', 
                 side='right', orientation='vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '$0,.0f'], 
                 labels = ['Strategy', 'NTrades', 
                           'PnL'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure9_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure9_2.title = '2019 - Strategies by Number of Trades and Closed PnL (size by PnL)'

#Figure layout (except size attributes, defined on section 4) 
figure9_2.title_style = {'font-size': '13px'}
figure9_2.background_style = {'fill': 'white'} 
figure9_2.animation_duration = 100

figure9_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure9_2.layout.width = '48%' #98%
figure9_2.layout.left = '20px'
figure9_2.layout.height = '900px'
figure9_2.legend_location = 'top-left'

In [49]:
HBox([figure9, figure9_2])

## Avg Dv01 per trade and Number of Trades per Strategy and Subclass (combined)

In [50]:
### Data
#Average Dv01 per trade, per strategy df
AvgDV01_byTicker = (pd.DataFrame(data = np.array([mean(abs(Dfs_byTicker[Ticker]['TotalDV01'])) for 
                                                 Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['AvgDV01']))

#Average dv01 per trade top performing strategies (by number of trades & PnL)
AvgDV01_TopPerfStr = AvgDV01_byTicker.loc[NTrades_TopPerfStr.index]

In [51]:
#Average Dv01 per trade, per strategy df
AvgDV01_byTicker_2 = (pd.DataFrame(data = np.array([mean(abs(Dfs_byTicker_2[Ticker]['TotalDV01'])) for 
                                                 Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['AvgDV01']))

#Average dv01 per trade top performing strategies (by number of trades & PnL)
AvgDV01_TopPerfStr_2 = AvgDV01_byTicker_2.loc[NTrades_TopPerfStr_2.index]

In [52]:
#Plot data
############################################################
Scatt_x_data = np.array(NTrades_byTicker['NTrades'])
Scatt_y_data = np.array(AvgDV01_byTicker['AvgDV01'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(NTrades_TopPerfStr['NTrades'])
Label_y_data = np.array(AvgDV01_TopPerfStr['AvgDV01'])
Label_data = NTrades_TopPerfStr.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Trades',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'Average DV01 per trade', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '$0,.0f'], 
                 labels = ['Strategy', 'NTrades', 
                           'DV01'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure10 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure10.title = '2018 - Strategies by Number of Trades and Avg DV01 per Trade (size by Dv01)'

#Figure layout (except size attributes, defined on section 4) 
figure10.title_style = {'font-size': '13px'}
figure10.background_style = {'fill': 'white'} 
figure10.animation_duration = 100

figure10.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure10.layout.width = '48%' #98%
figure10.layout.height = '900px'
figure10.legend_location = 'top-left'

In [53]:
#Plot data
############################################################
Scatt_x_data = np.array(NTrades_byTicker_2['NTrades'])
Scatt_y_data = np.array(AvgDV01_byTicker_2['AvgDV01'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(NTrades_TopPerfStr_2['NTrades'])
Label_y_data = np.array(AvgDV01_TopPerfStr_2['AvgDV01'])
Label_data = NTrades_TopPerfStr_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Trades',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'Average DV01 per trade', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '$0,.0f'], 
                 labels = ['Strategy', 'NTrades', 
                           'DV01'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure10_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure10_2.title = '2019 - Strategies by Number of Trades and Avg DV01 per Trade (size by Dv01)'

#Figure layout (except size attributes, defined on section 4) 
figure10_2.title_style = {'font-size': '13px'}
figure10_2.background_style = {'fill': 'white'} 
figure10_2.animation_duration = 100

figure10_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure10_2.layout.width = '48%' #98%
figure10_2.layout.left = '20px'
figure10_2.layout.height = '900px'
figure10_2.legend_location = 'top-left'

In [54]:
HBox([figure10, figure10_2])

## Avg closed PnL per trade and Avg DV01 per trade per Strategy and Subclass (combined)

In [55]:
### Data

#Average Dv01 per trade, per strategy df
AvgPnL_byTicker = (pd.DataFrame(data = np.array([mean(Dfs_byTicker[Ticker]['NetPnL']) for 
                                                 Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['NetPnL']))

#Average dv01 per trade top performing strategies (by number of trades & PnL)
AvgPnL_TopPerfStr = AvgPnL_byTicker.loc[NTrades_TopPerfStr.index]

AvgPnL_TopPerfStr_b = AvgPnL_byTicker.nlargest(15, columns = 'NetPnL')
AvgDV01_TopPerfStr_b = AvgDV01_byTicker.loc[AvgPnL_TopPerfStr_b.index]

In [56]:
#Average Dv01 per trade, per strategy df
AvgPnL_byTicker_2 = (pd.DataFrame(data = np.array([mean(Dfs_byTicker_2[Ticker]['NetPnL']) for 
                                                 Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['NetPnL']))

#Average dv01 per trade top performing strategies (by number of trades & PnL)
AvgPnL_TopPerfStr_2 = AvgPnL_byTicker_2.loc[NTrades_TopPerfStr_2.index]

AvgPnL_TopPerfStr_b_2 = AvgPnL_byTicker_2.nlargest(15, columns = 'NetPnL')
AvgDV01_TopPerfStr_b_2 = AvgDV01_byTicker_2.loc[AvgPnL_TopPerfStr_b_2.index]

In [57]:
#Plot data
############################################################
Scatt_x_data = np.array(AvgDV01_byTicker['AvgDV01'])
Scatt_y_data = np.array(AvgPnL_byTicker['NetPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])

Label_x_data = np.array(AvgDV01_TopPerfStr_b['AvgDV01'])
Label_y_data = np.array(AvgPnL_TopPerfStr_b['NetPnL'])
Label_data = AvgPnL_TopPerfStr_b.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '$0,.0f', 
            label_location = 'end', 
            label = 'Average DV01 per trade',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'Average PnL per trade', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '$0,.0f', '$0,.0f'], 
                 labels = ['Strategy', 'DV01', 
                           'PnL'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure11 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure11.title = '2018 - Strategies by Avg DV01 per Trade and Avg PnL per Trade (size by PnL)'

#Figure layout (except size attributes, defined on section 4) 
figure11.title_style = {'font-size': '13px'}
figure11.background_style = {'fill': 'white'} 
figure11.animation_duration = 100

figure11.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure11.layout.width = '48%' #98%
figure11.layout.height = '900px'
figure11.legend_location = 'top-left'

In [58]:
#Plot data
############################################################

Scatt_x_data = np.array(AvgDV01_byTicker_2.drop('U9V9X9')['AvgDV01'])
Scatt_y_data = np.array(AvgPnL_byTicker_2.drop('U9V9X9')['NetPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2.drop('U9V9X9')['SubClass'])

Label_x_data = np.array(AvgDV01_TopPerfStr_b_2['AvgDV01'])
Label_y_data = np.array(AvgPnL_TopPerfStr_b_2['NetPnL'])
Label_data = AvgPnL_TopPerfStr_b_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '$0,.0f', 
            label_location = 'end', 
            label = 'Average DV01 per trade',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '$0,.0f', 
            label_location = 'end',
            label = 'Average PnL per trade', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                size = Scatt_y_data,
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False,                 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y, 
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '$0,.0f', '$0,.0f'], 
                 labels = ['Strategy', 'DV01', 
                           'PnL'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure11_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure11_2.title = '2019 - Strategies by Avg DV01 per Trade and Avg PnL per Trade (size by PnL)'

#Figure layout (except size attributes, defined on section 4) 
figure11_2.title_style = {'font-size': '13px'}
figure11_2.background_style = {'fill': 'white'} 
figure11_2.animation_duration = 100

figure11_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure11_2.layout.width = '48%' #98%
figure11_2.layout.left = '20px'
figure11_2.layout.height = '900px'
figure11_2.legend_location = 'top-left'

In [59]:
HBox([figure11, figure11_2])

## Number of Longs and Shorts per Strategy and Subclass (combined)

In [60]:
### Data
#Number of long contracts (total sum), per strategy df
Longs_byTicker = (pd.DataFrame(data = np.array([(sum(Dfs_byTicker[Ticker][
                                                  'TotalQty'][Dfs_byTicker[
                                                  Ticker]['TotalQty'] > 0])) for 
                                                 Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['TotalQty']))

#Number of short contracts (total sum), per strategy df
Shorts_byTicker = (pd.DataFrame(data = np.array([(abs(sum(Dfs_byTicker[Ticker][
                                                  'TotalQty'][Dfs_byTicker[
                                                  Ticker]['TotalQty'] < 0]))) for 
                                                 Ticker in TradedStrategies]), 
                                 index = TradedStrategies, columns = ['TotalQty']))

#Longs ratio to total number of contracts traded, per strategy df
LongsR_byTicker = Longs_byTicker / (Longs_byTicker + Shorts_byTicker)

#Number of long & short contracts (total sum), 
#per trade top performing strategies (by number of trades & PnL)
Longs_TopPerfStr = Longs_byTicker.loc[NTrades_TopPerfStr.index]
Shorts_TopPerfStr = Shorts_byTicker.loc[NTrades_TopPerfStr.index]

LongsR_TopPerfStr = LongsR_byTicker.loc[NTrades_TopPerfStr.index]

In [61]:
#Number of long contracts (total sum), per strategy df
Longs_byTicker_2 = (pd.DataFrame(data = np.array([(sum(Dfs_byTicker_2[Ticker][
                                                  'TotalQty'][Dfs_byTicker_2[
                                                  Ticker]['TotalQty'] > 0])) for 
                                                 Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['TotalQty']))

#Number of short contracts (total sum), per strategy df
Shorts_byTicker_2 = (pd.DataFrame(data = np.array([(abs(sum(Dfs_byTicker_2[Ticker][
                                                  'TotalQty'][Dfs_byTicker_2[
                                                  Ticker]['TotalQty'] < 0]))) for 
                                                 Ticker in TradedStrategies_2]), 
                                 index = TradedStrategies_2, columns = ['TotalQty']))

#Longs ratio to total number of contracts traded, per strategy df
LongsR_byTicker_2 = Longs_byTicker_2 / (Longs_byTicker_2 + Shorts_byTicker_2)

#Number of long & short contracts (total sum), 
#per trade top performing strategies (by number of trades & PnL)
Longs_TopPerfStr_2 = Longs_byTicker_2.loc[NTrades_TopPerfStr_2.index]
Shorts_TopPerfStr_2 = Shorts_byTicker_2.loc[NTrades_TopPerfStr_2.index]

LongsR_TopPerfStr_2 = LongsR_byTicker_2.loc[NTrades_TopPerfStr_2.index]

In [62]:
#Plot data
############################################################
Scatt_x_data = np.array(Longs_byTicker['TotalQty'])
Scatt_y_data = np.array(Shorts_byTicker['TotalQty'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Longs_TopPerfStr['TotalQty'])
Label_y_data = np.array(Shorts_TopPerfStr['TotalQty'])
Label_data = NTrades_TopPerfStr.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])
#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Longs',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '0f', 
            label_location = 'end',
            label = 'Number of Shorts', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False,                 
                size = NTrades_byTicker['NTrades'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '0f'], 
                 labels = ['Strategy', 'Longs', 
                           'Shorts'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure12 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure12.title = '2018 - Strategies by Number of Longs and Shorts (size by Total Nº of Trades)'

#Figure layout (except size attributes, defined on section 4) 
figure12.title_style = {'font-size': '13px'}
figure12.background_style = {'fill': 'white'} 
figure12.animation_duration = 100

figure12.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure12.layout.width = '48%' #98%
#figure12.layout.left = '20px'
figure12.layout.height = '900px'
figure12.legend_location = 'top-left'

In [63]:
#Plot data
############################################################
Scatt_x_data = np.array(Longs_byTicker_2['TotalQty'])
Scatt_y_data = np.array(Shorts_byTicker_2['TotalQty'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Longs_TopPerfStr_2['TotalQty'])
Label_y_data = np.array(Shorts_TopPerfStr_2['TotalQty'])
Label_data = NTrades_TopPerfStr_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Longs',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '0f', 
            label_location = 'end',
            label = 'Number of Shorts', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False,                 
                size = NTrades_byTicker_2['NTrades'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 4500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '0f'], 
                 labels = ['Strategy', 'Longs', 
                           'Shorts'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure12_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure12_2.title = '2019 - Strategies by Number of Longs and Shorts (size by Total Nº of Trades)'

#Figure layout (except size attributes, defined on section 4) 
figure12_2.title_style = {'font-size': '13px'}
figure12_2.background_style = {'fill': 'white'} 
figure12_2.animation_duration = 100

figure12_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure12_2.layout.width = '48%' #98%
figure12_2.layout.left = '20px'
figure12_2.layout.height = '900px'
figure12_2.legend_location = 'top-left'

In [64]:
HBox([figure12, figure12_2])

## Avg Bps and Number of Trades for Longs and Shorts, per Strategy and Subclass (combined)

In [65]:
### Data
#Avg bps of profit of long contracts, per strategy df
AvgBpsLongs_byTicker = (pd.DataFrame(data = np.array([(mean(Dfs_byTicker[Ticker][
                                                      'BpsPnL'][Dfs_byTicker[
                                                      Ticker]['TotalQty'] > 0])) for 
                                                      Ticker in TradedStrategies]), 
                                     index = TradedStrategies, columns = ['BpsPnL']))

#Avg bps of profit of short contracts, per strategy df
AvgBpsShorts_byTicker = (pd.DataFrame(data = np.array([(mean(Dfs_byTicker[Ticker][
                                                      'BpsPnL'][Dfs_byTicker[
                                                      Ticker]['TotalQty'] < 0])) for 
                                                      Ticker in TradedStrategies]), 
                                      index = TradedStrategies, columns = ['BpsPnL']))

#Avg bps of profit of long & short contracts (total sum), 
#per trade top performing strategies (by number of trades & PnL)
AvgBpsLongs_TopPerfStr = AvgBpsLongs_byTicker.loc[NTrades_TopPerfStr.index]
AvgBpsShorts_TopPerfStr = AvgBpsShorts_byTicker.loc[NTrades_TopPerfStr.index]

In [66]:
#Avg bps of profit of long contracts, per strategy df
AvgBpsLongs_byTicker_2 = (pd.DataFrame(data = np.array([(mean(Dfs_byTicker_2[Ticker][
                                                      'BpsPnL'][Dfs_byTicker_2[
                                                      Ticker]['TotalQty'] > 0])) for 
                                                      Ticker in TradedStrategies_2]), 
                                     index = TradedStrategies_2, columns = ['BpsPnL']))

#Avg bps of profit of short contracts, per strategy df
AvgBpsShorts_byTicker_2 = (pd.DataFrame(data = np.array([(mean(Dfs_byTicker_2[Ticker][
                                                      'BpsPnL'][Dfs_byTicker_2[
                                                      Ticker]['TotalQty'] < 0])) for 
                                                      Ticker in TradedStrategies_2]), 
                                      index = TradedStrategies_2, columns = ['BpsPnL']))

#Avg bps of profit of long & short contracts (total sum), 
#per trade top performing strategies (by number of trades & PnL)
AvgBpsLongs_TopPerfStr_2 = AvgBpsLongs_byTicker_2.loc[NTrades_TopPerfStr_2.index]
AvgBpsShorts_TopPerfStr_2 = AvgBpsShorts_byTicker_2.loc[NTrades_TopPerfStr_2.index]

In [67]:
#Plot data
############################################################
Scatt_x_data = np.array(Longs_byTicker['TotalQty'])
Scatt_y_data = np.array(AvgBpsLongs_byTicker['BpsPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Longs_TopPerfStr['TotalQty'])
Label_y_data = np.array(AvgBpsLongs_TopPerfStr['BpsPnL'])
Label_data = NTrades_TopPerfStr.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Longs',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.2f', 
            label_location = 'end',
            label = 'Avg Bps of Profit Longs', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False, 
                size = Longs_byTicker['TotalQty'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 3500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '.2f'], 
                 labels = ['Strategy', 'Longs', 
                           'Avg PnL (bps)'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure13 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure13.title = '2018 - LONGS - Strategies by Avg Bps of Profit per Trade and Number of Longs (size by Nº of Longs)'

#Figure layout (except size attributes, defined on section 4) 
figure13.title_style = {'font-size': '13px'}
figure13.background_style = {'fill': 'white'} 
figure13.animation_duration = 100

figure13.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure13.layout.width = '48%' #98%
figure13.layout.height = '900px'
figure13.legend_location = 'top-left'

In [68]:
#Plot data
############################################################
Scatt_x_data = np.array(Shorts_byTicker['TotalQty'])
Scatt_y_data = np.array(AvgBpsShorts_byTicker['BpsPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Shorts_TopPerfStr['TotalQty'])
Label_y_data = np.array(AvgBpsShorts_TopPerfStr['BpsPnL'])
Label_data = NTrades_TopPerfStr.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Shorts',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.2f', 
            label_location = 'end',
            label = 'Avg Bps of Profit Shorts', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies, 
                display_names = False,                 
                size = Shorts_byTicker['TotalQty'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 3500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '.2f'], 
                 labels = ['Strategy', 'Shorts', 
                           'Avg PnL (bps)'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure14 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure14.title = '2018 - SHORTS - Strategies by Avg Bps of Profit per Trade and Number of Shorts (size by Nº of Shorts)'

#Figure layout (except size attributes, defined on section 4) 
figure14.title_style = {'font-size': '13px'}
figure14.background_style = {'fill': 'white'} 
figure14.animation_duration = 100

figure14.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure14.layout.width = '48%' #98%
figure14.layout.left = '20px'
figure14.layout.height = '900px'
figure14.legend_location = 'top-left'

In [69]:
#Plot data
############################################################
Scatt_x_data = np.array(Longs_byTicker_2['TotalQty'])
Scatt_y_data = np.array(AvgBpsLongs_byTicker_2['BpsPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Longs_TopPerfStr_2['TotalQty'])
Label_y_data = np.array(AvgBpsLongs_TopPerfStr_2['BpsPnL'])
Label_data = NTrades_TopPerfStr_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Longs',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.2f', 
            label_location = 'end',
            label = 'Avg Bps of Profit Longs', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False, 
                size = Longs_byTicker_2['TotalQty'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 3500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '.2f'], 
                 labels = ['Strategy', 'Longs', 
                           'Avg PnL (bps)'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure13_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure13_2.title = '2019 - LONGS - Strategies by Avg Bps of Profit per Trade and Number of Longs (size by Nº of Longs)'

#Figure layout (except size attributes, defined on section 4) 
figure13_2.title_style = {'font-size': '13px'}
figure13_2.background_style = {'fill': 'white'} 
figure13_2.animation_duration = 100

figure13_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure13_2.layout.width = '48%' #98%
figure13_2.layout.height = '900px'
figure13_2.legend_location = 'top-left'

In [70]:
#Plot data
############################################################
Scatt_x_data = np.array(Shorts_byTicker_2['TotalQty'])
Scatt_y_data = np.array(AvgBpsShorts_byTicker_2['BpsPnL'])
Scatt_ordinal_data = np.array(SubClass_byTicker_2['SubClass'])


#Best performing trades (labelled)
Label_x_data = np.array(Shorts_TopPerfStr_2['TotalQty'])
Label_y_data = np.array(AvgBpsShorts_TopPerfStr_2['BpsPnL'])
Label_data = NTrades_TopPerfStr_2.index.tolist()
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = LinearScale()
Scatt_y = LinearScale()
Scatt_s = LinearScale()
Scatt_o = OrdinalColorScale(colors=['#0070C0', '#3CDBCC', 
                                    '#01B0F0', '#31DC8D', 
                                    'gray', '#DE2369'])

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '0f', 
            label_location = 'end', 
            label = 'Number of Shorts',
            tick_style = {'font-size': '13px'}, 
            label_offset = '50px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.2f', 
            label_location = 'end',
            label = 'Avg Bps of Profit Shorts', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '13px'}, 
            label_offset = '60px')

#Color axis
ax_c = ColorAxis(scale = Scatt_o, label = 'Class', 
                 side = 'right', orientation = 'vertical')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data, 
                color = Scatt_ordinal_data,
                names = TradedStrategies_2, 
                display_names = False,                 
                size = Shorts_byTicker_2['TotalQty'], 
                scales = {'x': Scatt_x, 
                          'y': Scatt_y,
                          'size': Scatt_s,
                          'color': Scatt_o},
                default_size = 3500,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})


LabelsPlot = Label(x = Label_x_data, y = Label_y_data, 
                   text = Label_data, 
                   scales = {'x': Scatt_x, 'y': Scatt_y},
                   default_size = 14, font_weight = 'normal', 
                   colors =  ['black'], align = 'middle',
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['name', 'x', 'y'], 
                 formats = ['', '0f', '.2f'], 
                 labels = ['Strategy', 'Shorts', 
                           'Avg PnL (bps)'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure14_2 = Figure(marks = [Scatt, LabelsPlot], 
                 axes = [ax_x, ax_y, ax_c])

#Figure Title
figure14_2.title = '2019 - SHORTS - Strategies by Avg Bps of Profit per Trade and Number of Shorts (size by Nº of Shorts)'

#Figure layout (except size attributes, defined on section 4) 
figure14_2.title_style = {'font-size': '13px'}
figure14_2.background_style = {'fill': 'white'} 
figure14_2.animation_duration = 100

figure14_2.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure14_2.layout.width = '48%' #98%
figure14_2.layout.left = '20px'
figure14_2.layout.height = '900px'
figure14_2.legend_location = 'top-left'

In [71]:
HBox([figure13, figure13_2])

In [72]:
HBox([figure14, figure14_2])

In [73]:
# ## PnL, Margin (Closed, open, total) Overview

In [74]:
#Identify unique dates on the trades data frame
ClosedPnLDays = unique([str(FFsTrades.index[i]) for i in range(0, len(FFsTrades))])

#Total NET closed pnl per day
ClosedPnL = np.array([sum(FFsTrades[ithDate]['NetPnL']) for ithDate in ClosedPnLDays])

#Closed pnl data frame
ClosedPnLDf = pd.DataFrame(data = ClosedPnL, index = unique(FFsTrades.index), 
                           columns = ['ClosedPnL'])

#Merge Closed pnl with the Pnl Data frame (some dates are missing, when closed pnl was = 0)
PnLDf = pd.merge(CLXDataList['PnL'], ClosedPnLDf, how = 'outer', 
                 left_index = True, right_index = True)
PnLDf = PnLDf.fillna(value = 0)

#Add Open pnl to the pnl data frame
PnLDf['OpenPnL'] = PnLDf['PnL'] - PnLDf['ClosedPnL']

#Cumulative open & closed pnl
CumPnLDf = PnLDf.loc[:, ['PnL', 'ClosedPnL', 'OpenPnL']].cumsum()
CumPnLDf.columns = ['Acum_Closed', 'Acum_Open', 'Acum_PnL']

PnLDf['Acum_Closed'] = CumPnLDf['Acum_Closed']
PnLDf['Acum_Open'] = CumPnLDf['Acum_Open']
PnLDf = PnLDf.drop(labels = 'Acum_PnL', axis = 1)
PnLDf['Acum_PnL'] = CumPnLDf['Acum_PnL']

In [75]:
#Plot data
############################################################
x_data = CLXDataList['PnL'].index
y_data = np.array(CLXDataList['PnL']['Acum_PnL'])
y2_data = np.array(CLXDataList['PnL']['Margin'])
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 
MainTs_y2_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '13px'}, 
                   num_ticks = 10, tick_format = '%d/%m/%Y')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '$0,.0f', 
                   tick_style = {'font-size': '13px'},
                   grid_color = '#EDEDED')
 
#Y-Axis II
MainTs_y2_ax = Axis(scale = MainTs_y2_sc, 
                    orientation = 'vertical', 
                    side = 'right',
                    tick_format = '$0,.0f', 
                    tick_style = {'font-size': '13px'},
                    grid_color = 'white')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['Acumulated Daily PnL'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0'], 
               stroke_width = 2, 
               visible = True, 
               display_legend = True)

TsPlot2 = Lines(x = x_data, 
                y = y2_data, 
                labels = ['Daily Margin'], 
                scales = {'x': MainTs_x_dates, 
                          'y' : MainTs_y2_sc},
                colors = ['#3CDBCC'],
                stroke_width = 2, 
                opacities = [0.3], 
                visible = True, 
                display_legend = True, 
                fill = 'bottom', 
                fill_colors = ['#3CDBCC'], 
                fill_opacities = [0.1])

PointsScatt = Scatter(x = np.array(x_data), 
                      y = y_data,  
                      scales = {'x': MainTs_x_dates, 
                                'y' : MainTs_y_sc},                      
                     padding = 0.25, colors = ['#DE2369'], 
                     default_size = 60, 
                     default_opacities = [0.6], 
                     unhovered_style = {'opacity': 0.2}, 
                     hovered_style = {'opacity': 0.8})
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['%d/%m/%Y', '$0,.0f'], 
         labels = ['Date', 'PnL'])

#Set up the tooltip interaction 
PointsScatt.tooltip = def_tt
PointsScatt.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure15 = Figure(marks = [TsPlot, TsPlot2, PointsScatt], 
                 axes = [MainTs_x_ax, MainTs_y_ax, MainTs_y2_ax])

#Figure Title
figure15.title = 'CLX Acumulated Daily PnL (Left) & Daily Margin (Right)'

#Figure layout (except size attributes, defined on section 4)
figure15.title_style = {'font-size': '13px'}
figure15.background_style = {'fill': 'white'}
figure15.animation_duration = 800 
    
figure15.fig_margin = {'top':40, 'bottom':60, 'left':120, 'right':120}
figure15.layout.width = '98%'
figure15.layout.height = '700px'
figure15.legend_location = 'top-left'

In [76]:
#figure15

In [77]:
#Plot data
############################################################
y_data = np.array(CLXDataList['PnL']['PnL'])
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'})

#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
            orientation = 'vertical', 
            tick_style = {'font-size': '11px'}, 
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['Net PnL'],
                display_legend = False)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.9] * HistPlot.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '$0,.0f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure16 = Figure(marks = [HistPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure16.title = 'Daily PnL'

#Figure layout (except size attributes, defined on section 4)
figure16.title_style = {'font-size': '13px'}
figure16.background_style = {'fill': 'white'} 
figure16.animation_duration = 800

figure16.fig_margin = {'top':40, 'bottom':20, 'left':20, 'right':0}
figure16.layout.width = '30%'
figure16.layout.left = '0px'
figure16.layout.height = '450px'

In [78]:
#Plot data
############################################################
y_data = np.array(PnLDf['ClosedPnL'])
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'})

#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
            orientation = 'vertical', 
            tick_style = {'font-size': '11px'}, 
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#3CDBCC'], 
                stroke = '#D8D8D8',
                labels = ['Net PnL'],
                display_legend = False)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.9] * HistPlot.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '$0,.0f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure17 = Figure(marks = [HistPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure17.title = 'Daily Closed PnL'

#Figure layout (except size attributes, defined on section 4)
figure17.title_style = {'font-size': '13px'}
figure17.background_style = {'fill': 'white'} 
figure17.animation_duration = 800

figure17.fig_margin = {'top':40, 'bottom':20, 'left':20, 'right':0}
figure17.layout.width = '30%'
figure17.layout.left = '60px'
figure17.layout.height = '450px'

In [79]:
#Plot data
############################################################
y_data = np.array(PnLDf['OpenPnL'])
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
            tick_format = '$0,.0f', 
            tick_style = {'font-size': '11px'})

#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
            orientation = 'vertical', 
            tick_style = {'font-size': '11px'}, 
            grid_color = '#EDEDED')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Net PnL'],
                display_legend = False)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.9] * HistPlot.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '$0,.0f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure18 = Figure(marks = [HistPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure18.title = 'Daily Open PnL'

#Figure layout (except size attributes, defined on section 4)
figure18.title_style = {'font-size': '13px'}
figure18.background_style = {'fill': 'white'} 
figure18.animation_duration = 800

figure18.fig_margin = {'top':40, 'bottom':20, 'left':20, 'right':0}
figure18.layout.width = '30%'
figure18.layout.left = '120px'
figure18.layout.height = '450px'

In [80]:
#HBox([figure16, figure17, figure18])